In [93]:
import os
import re
import shutil

import pandas as pd
from python_pdb.parsers import parse_pdb_to_pandas
from python_pdb.entities import Structure

In [2]:
DATA_DIR = '/project/koohylab/bmcmaste/projects/tcr-pmhc-interface-analysis/data'

# Loading Datasets

Create pandas dataframes for datasets of apo-holo structures on both the TCR and pMHC point of view.

## Loading pMHC apo-holo information

In [41]:
pmhc_apo_holo = pd.read_json(os.path.join(DATA_DIR, 'apo_holo_mhc_structures.json'), orient='index').reset_index(names='mhc_group')

pmhc_apo_holo = pmhc_apo_holo.explode('holo').explode('apo')

pmhc_apo_holo[['holo_pdb_id', 'holo_resolution']] = pmhc_apo_holo['holo'].apply(pd.Series)
pmhc_apo_holo[['apo_pdb_id', 'apo_resolution']] = pmhc_apo_holo['apo'].apply(pd.Series)[['pdb_code', 'resolution']]

pmhc_apo_holo = pmhc_apo_holo.drop(['apo', 'holo'], axis=1)
pmhc_apo_holo = pmhc_apo_holo.dropna()

pmhc_apo_holo

,mhc_group,allele,peptide,holo_pdb_id,holo_resolution,apo_pdb_id,apo_resolution
1,hla_a_24_02_rypltfgw,hla_a_24_02,RYPLTFGW,8gvi,3.30,5hgb,2.40
1,hla_a_24_02_rypltfgw,hla_a_24_02,RYPLTFGW,8gvi,3.30,4wu5,2.40
1,hla_a_24_02_rypltfgw,hla_a_24_02,RYPLTFGW,8gvb,3.20,5hgb,2.40
1,hla_a_24_02_rypltfgw,hla_a_24_02,RYPLTFGW,8gvb,3.20,4wu5,2.40
2,hla_a_24_02_rfpltfgw,hla_a_24_02,RFPLTFGW,8gvg,3.37,5hga,2.20
...,...,...,...,...,...,...,...
113,h2_kb_eqykfysv,h2_kb,EQYKFYSV,2ckb,3.00,1lek,2.15
113,h2_kb_eqykfysv,h2_kb,EQYKFYSV,2ckb,3.00,1leg,1.75
114,h2_kb_kvitfidl,h2_kb,KVITFIDL,1kj2,2.71,1kj3,2.30
116,h2_kb_infdfnti,h2_kb,INFDFNTI,1fo0,2.50,2clz,1.90


In [42]:
pmhc_apo_holo = pd.concat([pmhc_apo_holo[['mhc_group', 'allele', 'peptide', 'holo_pdb_id', 'holo_resolution']],
           pmhc_apo_holo[['mhc_group', 'allele', 'peptide', 'apo_pdb_id', 'apo_resolution']]]).reset_index(drop=True)
pmhc_apo_holo

,mhc_group,allele,peptide,holo_pdb_id,holo_resolution,apo_pdb_id,apo_resolution
0,hla_a_24_02_rypltfgw,hla_a_24_02,RYPLTFGW,8gvi,3.30,NaN,NaN
1,hla_a_24_02_rypltfgw,hla_a_24_02,RYPLTFGW,8gvi,3.30,NaN,NaN
2,hla_a_24_02_rypltfgw,hla_a_24_02,RYPLTFGW,8gvb,3.20,NaN,NaN
3,hla_a_24_02_rypltfgw,hla_a_24_02,RYPLTFGW,8gvb,3.20,NaN,NaN
4,hla_a_24_02_rfpltfgw,hla_a_24_02,RFPLTFGW,8gvg,3.37,NaN,NaN
...,...,...,...,...,...,...,...
543,h2_kb_eqykfysv,h2_kb,EQYKFYSV,NaN,NaN,1lek,2.15
544,h2_kb_eqykfysv,h2_kb,EQYKFYSV,NaN,NaN,1leg,1.75
545,h2_kb_kvitfidl,h2_kb,KVITFIDL,NaN,NaN,1kj3,2.30
546,h2_kb_infdfnti,h2_kb,INFDFNTI,NaN,NaN,2clz,1.90


In [43]:
pmhc_apo_holo['state'] = pmhc_apo_holo.apply(lambda row: 'holo' if not pd.isna(row['holo_pdb_id']) else 'apo', axis=1)

pmhc_apo_holo['pdb_id'] = pmhc_apo_holo.apply(lambda row: row['holo_pdb_id'] if row['state'] == 'holo' else row['apo_pdb_id'], axis=1)
pmhc_apo_holo['resolution'] = pmhc_apo_holo.apply(lambda row: row['holo_resolution'] if row['state'] == 'holo' else row['apo_resolution'], axis=1)

pmhc_apo_holo = pmhc_apo_holo.drop(['holo_pdb_id', 'holo_resolution', 'apo_pdb_id', 'apo_resolution'], axis=1)
pmhc_apo_holo = pmhc_apo_holo.drop_duplicates()

pmhc_apo_holo

,mhc_group,allele,peptide,state,pdb_id,resolution
0,hla_a_24_02_rypltfgw,hla_a_24_02,RYPLTFGW,holo,8gvi,3.30
2,hla_a_24_02_rypltfgw,hla_a_24_02,RYPLTFGW,holo,8gvb,3.20
4,hla_a_24_02_rfpltfgw,hla_a_24_02,RFPLTFGW,holo,8gvg,3.37
6,hla_a_24_02_qlprlfpll,hla_a_24_02,QLPRLFPLL,holo,7nme,2.20
7,hla_a_24_02_qlprlfpll,hla_a_24_02,QLPRLFPLL,holo,7nmf,2.98
...,...,...,...,...,...,...
541,h2_kb_eqykfysv,h2_kb,EQYKFYSV,apo,1lek,2.15
542,h2_kb_eqykfysv,h2_kb,EQYKFYSV,apo,1leg,1.75
545,h2_kb_kvitfidl,h2_kb,KVITFIDL,apo,1kj3,2.30
546,h2_kb_infdfnti,h2_kb,INFDFNTI,apo,2clz,1.90


### Applying quality control

Screen out structures with worse than 3.50 Å resolution as this is what has been done for the TCR side of things.

In [44]:
pmhc_apo_holo['resolution'] = pmhc_apo_holo['resolution'].apply(float)
pmhc_apo_holo = pmhc_apo_holo.query('resolution < 3.50')

pmhc_apo_holo

,mhc_group,allele,peptide,state,pdb_id,resolution
0,hla_a_24_02_rypltfgw,hla_a_24_02,RYPLTFGW,holo,8gvi,3.30
2,hla_a_24_02_rypltfgw,hla_a_24_02,RYPLTFGW,holo,8gvb,3.20
4,hla_a_24_02_rfpltfgw,hla_a_24_02,RFPLTFGW,holo,8gvg,3.37
6,hla_a_24_02_qlprlfpll,hla_a_24_02,QLPRLFPLL,holo,7nme,2.20
7,hla_a_24_02_qlprlfpll,hla_a_24_02,QLPRLFPLL,holo,7nmf,2.98
...,...,...,...,...,...,...
541,h2_kb_eqykfysv,h2_kb,EQYKFYSV,apo,1lek,2.15
542,h2_kb_eqykfysv,h2_kb,EQYKFYSV,apo,1leg,1.75
545,h2_kb_kvitfidl,h2_kb,KVITFIDL,apo,1kj3,2.30
546,h2_kb_infdfnti,h2_kb,INFDFNTI,apo,2clz,1.90


## Load TCR apo-holo information

In [45]:
tcr_apo_holo = pd.read_json(os.path.join(DATA_DIR, 'apo-holo-mhc-class-I_refined', 'summary.json'), orient='index').reset_index(names='tcr_group')

tcr_apo_holo = tcr_apo_holo.melt(id_vars='tcr_group')
tcr_apo_holo = tcr_apo_holo.join(tcr_apo_holo['value'].apply(pd.Series)).query('value.notnull()')

tcr_apo_holo = tcr_apo_holo.drop(['variable', 'value'], axis=1)

tcr_apo_holo

,tcr_group,pdb_id,state,alpha_chain,beta_chain,peptide_chain,mhc_chain
0,DRGSQS-IYSNGD-AVNFGGGKLI-MRHNA-SNTAGT-ASSLSFGTEAF,6am5,holo,D,E,C,A
1,DRGSQS-IYSNGD-AVNRDDKII-SEHNR-FQNEAQ-ASSPDIEQY,7rtr,holo,D,E,C,A
2,DRGSQS-IYSNGD-AVRTNSGYALN-QGHDT-YYEEEE-ASSDTVS...,5nmg,holo,D,E,C,A
3,DRGSQS-IYSNGD-AVTTDSWGKLQ-MNHEY-SVGAGI-ASRPGLA...,3pwp,holo,D,E,C,A
4,DRGSQS-IYSNGD-GTYNQGGKLI-MNHEY-SMNVEV-ASSGASHEQY,3w0w,holo,D,E,C,A
...,...,...,...,...,...,...,...
138,NSAFQY-TYSSGN-AMRGDSSYKLI-SGHDY-FNNNVP-ASSLWEK...,5c08,holo,D,E,C,A
147,DRGSQS-IYSNGD-AVTTDSWGKLQ-MNHEY-SVGAGI-ASRPGLA...,1ao7,holo,D,E,C,A
156,NSAFQY-TYSSGN-AMRGDSSYKLI-SGHDY-FNNNVP-ASSLWEK...,5c0b,holo,I,J,H,F
165,DRGSQS-IYSNGD-AVTTDSWGKLQ-MNHEY-SVGAGI-ASRPGLA...,3qfj,holo,D,E,C,A


# Collate and find overlap between these datasets

In [46]:
merged = pmhc_apo_holo.merge(tcr_apo_holo, how='outer', on=['pdb_id', 'state'])
merged

,mhc_group,allele,peptide,state,pdb_id,resolution,tcr_group,alpha_chain,beta_chain,peptide_chain,mhc_chain
0,hla_a_24_02_rypltfgw,hla_a_24_02,RYPLTFGW,holo,8gvi,3.30,NaN,NaN,NaN,NaN,NaN
1,hla_a_24_02_rypltfgw,hla_a_24_02,RYPLTFGW,holo,8gvb,3.20,NaN,NaN,NaN,NaN,NaN
2,hla_a_24_02_rfpltfgw,hla_a_24_02,RFPLTFGW,holo,8gvg,3.37,NaN,NaN,NaN,NaN,NaN
3,hla_a_24_02_qlprlfpll,hla_a_24_02,QLPRLFPLL,holo,7nme,2.20,NaN,NaN,NaN,NaN,NaN
4,hla_a_24_02_qlprlfpll,hla_a_24_02,QLPRLFPLL,holo,7nmf,2.98,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
323,NaN,NaN,NaN,apo,1tcr,NaN,YSATPY-YYSGDPVV-AVSGFASALT-NNHNN-SYGAGS-ASGGGGTLY,A,B,NaN,NaN
324,NaN,NaN,NaN,holo,1qsf,NaN,DRGSQS-IYSNGD-AVTTDSWGKLQ-MNHEY-SVGAGI-ASRPGLA...,D,E,C,A
325,NaN,NaN,NaN,holo,3d3v,NaN,DRGSQS-IYSNGD-AVTTDSWGKLQ-MNHEY-SVGAGI-ASRPGLA...,D,E,C,A
326,NaN,NaN,NaN,apo,3utp,NaN,NSAFQY-TYSSGN-AMRGDSSYKLI-SGHDY-FNNNVP-ASSLWEK...,D,E,NaN,NaN


In [47]:
holo_structures = merged.query("state == 'holo' and tcr_group.notnull() and mhc_group.notnull()").copy()

holo_structures['combined_name'] = holo_structures['mhc_group'] + '_' + holo_structures['tcr_group']
holo_structures = holo_structures.drop_duplicates()

holo_structures

,mhc_group,allele,peptide,state,pdb_id,resolution,tcr_group,alpha_chain,beta_chain,peptide_chain,mhc_chain,combined_name
12,hla_b_53_01_qasqevknw,hla_b_53_01,QASQEVKNW,holo,7r80,2.90,DSAIYN-IQSSQRE-AQLNQAGTALI-MNHEY-SVGAGI-ASSYGT...,A,B,E,C,hla_b_53_01_qasqevknw_DSAIYN-IQSSQRE-AQLNQAGTA...
15,hla_a_02_01_ylqprtfll,hla_a_02_01,YLQPRTFLL,holo,7rtr,2.60,DRGSQS-IYSNGD-AVNRDDKII-SEHNR-FQNEAQ-ASSPDIEQY,D,E,C,A,hla_a_02_01_ylqprtfll_DRGSQS-IYSNGD-AVNRDDKII-...
17,hla_a_02_01_ylqprtfll,hla_a_02_01,YLQPRTFLL,holo,7n1f,2.39,DRGSQS-IYSNGD-AVNRDDKII-SEHNR-FQNEAQ-ASSPDIEQY,D,E,C,A,hla_a_02_01_ylqprtfll_DRGSQS-IYSNGD-AVNRDDKII-...
22,hla_a_02_01_hmtevvrhc,hla_a_02_01,HMTEVVRHC,holo,6vrm,2.61,NSASQS-VYSSG-VVQPGGYQKVT-MNHNS-SASEGT-ASSEGLWQ...,D,E,P,A,hla_a_02_01_hmtevvrhc_NSASQS-VYSSG-VVQPGGYQKVT...
27,hla_a_02_01_rlqslqtyv,hla_a_02_01,RLQSLQTYV,holo,7n1e,2.30,YSGSPE-HISR-ALSGFNNAGNMLT-SGHAT-FQNNGV-ASSLGGA...,D,E,C,A,hla_a_02_01_rlqslqtyv_YSGSPE-HISR-ALSGFNNAGNML...
39,hla_a_02_01_llfgypvyv,hla_a_02_01,LLFGYPVYV,holo,1ao7,2.60,DRGSQS-IYSNGD-AVTTDSWGKLQ-MNHEY-SVGAGI-ASRPGLA...,D,E,C,A,hla_a_02_01_llfgypvyv_DRGSQS-IYSNGD-AVTTDSWGKL...
40,hla_a_02_01_eaagigiltv,hla_a_02_01,EAAGIGILTV,holo,6tmo,2.10,FLGSQS-TYREGD-AVNDGGRLT-GTSNPN-WGPFG-AWSETGLGM...,D,E,C,A,hla_a_02_01_eaagigiltv_FLGSQS-TYREGD-AVNDGGRLT...
56,hla_a_02_01_elagigiltv,hla_a_02_01,ELAGIGILTV,holo,4jff,2.43,FLGSQS-TYREGD-AVNDGGRLT-GTSNPN-WGPFG-AWSETGLGM...,D,E,C,A,hla_a_02_01_elagigiltv_FLGSQS-TYREGD-AVNDGGRLT...
57,hla_a_02_01_elagigiltv,hla_a_02_01,ELAGIGILTV,holo,3qdg,2.69,DRGSQS-IYSNGD-AVNFGGGKLI-MRHNA-SNTAGT-ASSLSFGTEAF,D,E,C,A,hla_a_02_01_elagigiltv_DRGSQS-IYSNGD-AVNFGGGKL...
62,hla_a_02_01_aagigiltv,hla_a_02_01,AAGIGILTV,holo,6eqb,2.81,FLGSQS-TYREGD-AVNDGGRLT-GTSNPN-WGPFG-AWSETGLGM...,D,E,C,A,hla_a_02_01_aagigiltv_FLGSQS-TYREGD-AVNDGGRLT-...


In [48]:
apo_mhcs = merged.query("state == 'apo' and mhc_group.isin(@holo_structures.mhc_group)").copy()
apo_mhcs = apo_mhcs.dropna(axis=1, how='all')
apo_mhcs

,mhc_group,allele,peptide,state,pdb_id,resolution
166,hla_b_53_01_qasqevknw,hla_b_53_01,QASQEVKNW,apo,7r7v,1.60
168,hla_a_02_01_ylqprtfll,hla_a_02_01,YLQPRTFLL,apo,7rtd,2.05
169,hla_a_02_01_ylqprtfll,hla_a_02_01,YLQPRTFLL,apo,7n1a,2.06
170,hla_a_02_01_ylqprtfll,hla_a_02_01,YLQPRTFLL,apo,7p3d,1.67
171,hla_a_02_01_ylqprtfll,hla_a_02_01,YLQPRTFLL,apo,7n6d,2.30
...,...,...,...,...,...,...
280,hla_b_44_05_eeylkawtf,hla_b_44_05,EEYLKAWTF,apo,3kpq,1.84
281,hla_b_44_05_eeylqafty,hla_b_44_05,EEYLQAFTY,apo,3kpp,1.90
284,hla_a_02_01_sllmwitqc,hla_a_02_01,SLLMWITQC,apo,1s9w,2.20
293,h2_kb_eqykfysv,h2_kb,EQYKFYSV,apo,1lek,2.15


In [49]:
apo_tcrs = merged.query("state == 'apo' and tcr_group.isin(@holo_structures.tcr_group)").copy()
apo_tcrs = apo_tcrs.dropna(axis=1, how='all')
apo_tcrs

,state,pdb_id,tcr_group,alpha_chain,beta_chain
299,apo,7r7z,DSAIYN-IQSSQRE-AQLNQAGTALI-MNHEY-SVGAGI-ASSYGT...,A,B
300,apo,3vxq,DSAIYN-IQSSQRE-AVRMDSSYKLI-SEHNR-FQNEAQ-ASSSWD...,D,E
301,apo,2pyf,DSAIYN-IQSSQRE-AVRPLLDGTYIPT-MNHEY-SVGAGT-ASSY...,A,B
302,apo,5iw1,NSAFDY-ILSVSNK-AASASFGDNSKLI-MSHET-SYDVDS-ASSL...,A,B
303,apo,6vth,NSASQS-VYSSG-VVQPGGYQKVT-MNHNS-SASEGT-ASSEGLWQ...,D,E
304,apo,2vlm,SVFSS-VVTGGEV-AGAGSQGNLI-LNHDA-SQIVND-ASSSRSSYEQY,D,E
306,apo,5nmd,DRGSQS-IYSNGD-AVRTNSGYALN-QGHDT-YYEEEE-ASSDTVS...,A,B
308,apo,6at6,NIATNDY-GYKTK-LVGEILDNFNKFY-MDHEN-SYDVKM-ASSQR...,A,B
309,apo,7n1c,YSGSPE-HISR-ALSGFNNAGNMLT-SGHAT-FQNNGV-ASSLGGA...,D,E
310,apo,3qeu,DRGSQS-IYSNGD-AVNFGGGKLI-MRHNA-SNTAGT-ASSLSFGTEAF,D,E


# Create new dataset with the collated TCR-pMHC apo-holo data

In [59]:
EXPORT_DIR = os.path.join(DATA_DIR, 'apo-holo TCR pMHC TCR-pMHC structures')

if not os.path.exists(EXPORT_DIR):
    os.mkdir(EXPORT_DIR)

In [100]:
end_pattern = re.compile(r'^END.*\n', re.MULTILINE)
ter_pattern = re.compile(r'^TER.*\n', re.MULTILINE)

for (mhc_name, tcr_name, group_name), group_holo in holo_structures.groupby(['mhc_group', 'tcr_group', 'combined_name']):
    print(group_name)
    
    group_apo_tcrs = apo_tcrs.query('tcr_group == @tcr_name')
    group_apo_mhcs = apo_mhcs.query('mhc_group == @mhc_name')

    group_path = os.path.join(EXPORT_DIR, group_name)

    if not os.path.exists(group_path):
        os.mkdir(group_path)
    
    # Holo Structures
    for _, holo_struct in group_holo.iterrows():
        chains = ''.join((holo_struct['alpha_chain'],
                          holo_struct['beta_chain'],
                          holo_struct['peptide_chain'],
                          holo_struct['mhc_chain']))

        base_path = f"{holo_struct['pdb_id']}_{chains}_holo.pdb"
        path = os.path.join(DATA_DIR,
                            'apo-holo-mhc-class-I_refined',
                            tcr_name,
                            base_path)
        
        shutil.copy2(path, os.path.join(EXPORT_DIR, group_name, f"tcr-pmhc_{holo_struct['pdb_id']}_{chains}.pdb"))
    
    # TCR Apo Structures
    for _, apo_struct in group_apo_tcrs.iterrows():
        chains = ''.join((apo_struct['alpha_chain'],
                          apo_struct['beta_chain']))

        base_path = f"{apo_struct['pdb_id']}_{chains}_apo.pdb"
        path = os.path.join(DATA_DIR,
                            'apo-holo-mhc-class-I_refined',
                            tcr_name,
                            base_path)
        
        shutil.copy2(path, os.path.join(EXPORT_DIR, group_name, f"tcr_{apo_struct['pdb_id']}_{chains}.pdb"))
    
    # MHC Apo Structures
    for _, apo_struct in group_apo_mhcs.iterrows():        
        path_abd = os.path.join(DATA_DIR,
                                'apo_vs_holo_pmhc_tcr/structures/apo_pmhc/clean',
                                f"{apo_struct['pdb_id']}_1_abd.pdb")
        path_peptide = os.path.join(DATA_DIR,
                                'apo_vs_holo_pmhc_tcr/structures/apo_pmhc/clean',
                                f"{apo_struct['pdb_id']}_1_peptide.pdb")
        
        with open(path_abd, 'r') as fh:
            mhc_abd_df = parse_pdb_to_pandas(fh.read())
        
        with open(path_peptide, 'r') as fh:
            peptide_df = parse_pdb_to_pandas(fh.read())
        
        mhc_abd_df['chain_id'] = 'A'
        peptide_df['chain_id'] = 'C'
        
        pmhc_df = pd.concat([mhc_abd_df, peptide_df])
        pmhc = Structure.from_pandas(pmhc_df)

        with open(os.path.join(EXPORT_DIR, group_name, f"pmhc_{apo_struct['pdb_id']}.pdb"), 'w') as fh:
            fh.write(str(pmhc))

h2_dd_rgpgrafvti_NSAFDY-ILSVSNK-AASASFGDNSKLI-MSHET-SYDVDS-ASSLGHTEVF
h2_kb_eqykfysv_YSATPY-YYSGDPVV-AVSGFASALT-NNHNN-SYGAGS-ASGGGGTLY
hla_a_02_01_aagigiltv_DRGSQS-IYSNGD-AVNFGGGKLI-MRHNA-SNTAGT-ASSLSFGTEAF


/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:509: StructureConstructionWarning: AtomRecord(atom_num=6, atom_name='CA', alt_loc='A', res_name='SER', chain_id='A', seq_id=2, insert_code=None, x_pos=-56.161, y_pos=46.758, z_pos=75.377, occupancy=0.5, b_factor=31.16, element='C', charge=None): ATOM contains alternate location. The `split_states` method can be used to separate these possible conformations into separate models.
  warnings.warn((f'{record}: ATOM contains alternate location. '
/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:509: StructureConstructionWarning: AtomRecord(atom_num=9, atom_name='CB', alt_loc='A', res_name='SER', chain_id='A', seq_id=2, insert_code=None, x_pos=-54.899, y_pos=46.173, z_pos=75.995, occupancy=0.5, b_factor=31.53, element='C', charge=None): ATOM contains alternate location. The `split_states` method can be used to separate these possible conformations into separate models.
  warnings.warn((f'{r

hla_a_02_01_aagigiltv_FLGSQS-TYREGD-AVNDGGRLT-GTSNPN-WGPFG-AWSETGLGMGGWQ
hla_a_02_01_alwgpdpaaa_NSAFQY-TYSSGN-AMRGDSSYKLI-SGHDY-FNNNVP-ASSLWEKLAKNIQY
hla_a_02_01_aqwgpdpaaa_NSAFQY-TYSSGN-AMRGDSSYKLI-SGHDY-FNNNVP-ASSLWEKLAKNIQY
hla_a_02_01_eaagigiltv_FLGSQS-TYREGD-AVNDGGRLT-GTSNPN-WGPFG-AWSETGLGMGGWQ


/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:509: StructureConstructionWarning: AtomRecord(atom_num=461, atom_name='N', alt_loc='A', res_name='GLU', chain_id='A', seq_id=58, insert_code=None, x_pos=-56.392, y_pos=68.293, z_pos=55.027, occupancy=0.5, b_factor=26.46, element='N', charge=None): ATOM contains alternate location. The `split_states` method can be used to separate these possible conformations into separate models.
  warnings.warn((f'{record}: ATOM contains alternate location. '
/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:509: StructureConstructionWarning: AtomRecord(atom_num=462, atom_name='CA', alt_loc='A', res_name='GLU', chain_id='A', seq_id=58, insert_code=None, x_pos=-55.429, y_pos=68.79, z_pos=56.029, occupancy=0.5, b_factor=25.97, element='C', charge=None): ATOM contains alternate location. The `split_states` method can be used to separate these possible conformations into separate models.
  warnings.warn((

hla_a_02_01_elaaigiltv_FLGSQS-TYREGD-AVNDGGRLT-GTSNPN-WGPFG-AWSETGLGMGGWQ
hla_a_02_01_elagigiltv_DRGSQS-IYSNGD-AVNFGGGKLI-MRHNA-SNTAGT-ASSLSFGTEAF
hla_a_02_01_elagigiltv_FLGSQS-TYREGD-AVNDGGRLT-GTSNPN-WGPFG-AWSETGLGMGGWQ
hla_a_02_01_gilgfvftl_SVFSS-VVTGGEV-AGAGSQGNLI-LNHDA-SQIVND-ASSSRSSYEQY
hla_a_02_01_gilglvftl_SVFSS-VVTGGEV-AGAGSQGNLI-LNHDA-SQIVND-ASSSRSSYEQY
hla_a_02_01_hmtevvrhc_NSASQS-VYSSG-VVQPGGYQKVT-MNHNS-SASEGT-ASSEGLWQVGDEQY
hla_a_02_01_lgygfvnyi_DRGSQS-IYSNGD-AVTTDSWGKLQ-MNHEY-SVGAGI-ASRPGLAGGRPEQY
hla_a_02_01_llfgypvyv_DRGSQS-IYSNGD-AVTTDSWGKLQ-MNHEY-SVGAGI-ASRPGLAGGRPEQY


...

/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:509: StructureConstructionWarning: AtomRecord(atom_num=666, atom_name='CA', alt_loc='A', res_name='ARG', chain_id='A', seq_id=82, insert_code=None, x_pos=-21.773, y_pos=54.991, z_pos=58.03, occupancy=0.5, b_factor=13.62, element='C', charge=None): ATOM contains alternate location. The `split_states` method can be used to separate these possible conformations into separate models.
  warnings.warn((f'{record}: ATOM contains alternate location. '
/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:509: StructureConstructionWarning: AtomRecord(atom_num=669, atom_name='CB', alt_loc='A', res_name='ARG', chain_id='A', seq_id=82, insert_code=None, x_pos=-21.786, y_pos=55.456, z_pos=56.559, occupancy=0.5, b_factor=15.96, element='C', charge=None): ATOM contains alternate location. The `split_states` method can be used to separate these possible conformations into separate models.
  warnings.warn(

hla_b_44_05_eeylkawtf_TISGTDY-GLTSN-ILPLAGGTSYGKLT-SGHVS-FQNEAQ-ASSLGQAYEQY
hla_b_44_05_eeylqafty_TISGTDY-GLTSN-ILPLAGGTSYGKLT-SGHVS-FQNEAQ-ASSLGQAYEQY


/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:509: StructureConstructionWarning: AtomRecord(atom_num=464, atom_name='N', alt_loc='A', res_name='GLU', chain_id='A', seq_id=58, insert_code=None, x_pos=-56.539, y_pos=68.193, z_pos=55.335, occupancy=0.5, b_factor=25.39, element='N', charge=None): ATOM contains alternate location. The `split_states` method can be used to separate these possible conformations into separate models.
  warnings.warn((f'{record}: ATOM contains alternate location. '
/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:509: StructureConstructionWarning: AtomRecord(atom_num=465, atom_name='CA', alt_loc='A', res_name='GLU', chain_id='A', seq_id=58, insert_code=None, x_pos=-55.488, y_pos=68.771, z_pos=56.16, occupancy=0.5, b_factor=24.99, element='C', charge=None): ATOM contains alternate location. The `split_states` method can be used to separate these possible conformations into separate models.
  warnings.warn((

hla_b_53_01_qasqevknw_DSAIYN-IQSSQRE-AQLNQAGTALI-MNHEY-SVGAGI-ASSYGTGINYGYT


/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:509: StructureConstructionWarning: AtomRecord(atom_num=22, atom_name='CA', alt_loc='A', res_name='SER', chain_id='A', seq_id=4, insert_code=None, x_pos=-51.975, y_pos=50.204, z_pos=70.664, occupancy=0.54, b_factor=17.14, element='C', charge=None): ATOM contains alternate location. The `split_states` method can be used to separate these possible conformations into separate models.
  warnings.warn((f'{record}: ATOM contains alternate location. '
/home/b/bmcmaste/.local/lib/python3.11/site-packages/python_pdb/entities.py:509: StructureConstructionWarning: AtomRecord(atom_num=25, atom_name='CB', alt_loc='A', res_name='SER', chain_id='A', seq_id=4, insert_code=None, x_pos=-50.759, y_pos=49.28, z_pos=70.695, occupancy=0.54, b_factor=22.72, element='C', charge=None): ATOM contains alternate location. The `split_states` method can be used to separate these possible conformations into separate models.
  warnings.warn((f